In [1]:
import sys


no_of_cpu =4
max_cores =4
executor_mem = '26g'


Job_Name = 'Mayur : cust_id check'

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests

# mesos_ip = 'mesos://10.35.12.205:5050'
mesos_ip = 'mesos://10.33.195.18:5050'
# mesos_ip = 'mesos://10.33.195.43:5050'
spark.stop() ############# NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL ###############################
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','120m')
conf.set('spark.sql.crossJoin.enabled', 'true')

# gpdb_env = 'prod'

# if gpdb_env=='uat':
#     gpdb_url = "jdbc:postgresql://10.33.195.103:5432/gpadmin"
#     gpdb_user = "gpspark"
#     gpdb_pass = "spark@456"
# else:
#     gpdb_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
#     gpdb_user = "gpspark"
#     gpdb_pass = "spark@456"



conf.set('spark.executor.heartbeatInterval','120s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx16g -Xms16g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
#23-11-2020
conf.set('spark.memory.offHeap.enabled','true')
conf.set('spark.memory.offHeap.size','16g')

conf.set('spark.sql.shuffle.partitions','80')
#conf.set('spark.sql.crossJoin.enabled', 'true')
# conf.set('es.nodes',es_nodes)
# conf.set('es.port',es_port)
# conf.set('es.nodes.wan.only','true')
# conf.set("spark.sql.autoBroadcastJoinThreshold",-1)
conf.set("spark.sql.broadcastTimeout",9000)
# config spark.scroll.keepalive("10m")
conf.set("spark.scroll.keepalive","120m")
#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
#conf.set('spark.num.executors','4')

# conf.set('spark.es.net.http.auth.user', es_user)
# conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)


# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


from pyspark.sql.window import *
from pyspark.sql.functions import row_number

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17342,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-01-25 06:40:25.218654'

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_progress(table_name,source):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime','records').filter(col('table_name')==table_name)\
            .filter(col('source')==source).filter(col('records')!=0)
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run)) MC:2018-11-01
        #ocrm 2017-12-28 icn:2019-04-05 10:58:49.291000  ICRM:2018-01-26 2023-07-29 00:00:00.000

    except Exception as E: 
        Max_last_run = datetime.datetime(2018,10,30, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
intermediary_df.createOrReplaceTempView('intermediary_table')

query1_mob = """SELECT txt_mobile mobile from intermediary_table
union
select txt_telephone from intermediary_table
union
select txt_telephone1 from intermediary_table
union
select txt_contact_phone from intermediary_table
union
select txt_emer_person_mobile_number from intermediary_table
union
select txt_policy_phone from intermediary_table
union
select txt_alt_mobile from intermediary_table"""

intermediary_df_mob = sqlContext.sql(query1_mob)




gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1160",
"dbschema": "staging",
"dbtable": "retention_auto_producer_contact"}
contact_prod = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr("contact as contact1")

agent_no=intermediary_df_mob.union(contact_prod)



agent_no = agent_no.withColumn("mobile", when((length(col("mobile")) == 12), 
                                               regexp_replace(col("mobile"), "^91", "")).otherwise(lit(col("mobile"))))

agent_no = agent_no.withColumn("mobile", when((length(col("mobile")) == 11),
                                               regexp_replace(col("mobile"), "^0", "")).otherwise(lit(col("mobile"))))

agent_no.persist(pyspark.StorageLevel.MEMORY_AND_DISK)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[mobile: string]

In [23]:
query1_email = """select txt_email from intermediary_table
union
select txt_alt_email_id from intermediary_table
union
select txt_email_sms_reqd from intermediary_table"""


intermediary_df_email = sqlContext.sql(query1_email)


intermediary_df_email.persist(pyspark.StorageLevel.MEMORY_AND_DISK)


gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1160",
"dbschema": "staging",
"dbtable": "retention_auto_producer_email"}

email_prod = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr("email as email1")

agent_email=intermediary_df_email.union(email_prod)


agent_email.persist(pyspark.StorageLevel.MEMORY_AND_DISK)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[txt_email: string]

In [24]:
condition = [lower(trim(contact_df.value))==lower(trim(agent_no.mobile))]

contact_df_x1 = contact_df.join(agent_no,condition,how="left")
##contact_df_recheck_x1 = contact_df_recheck.filter(col('value').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
contact_df_x1.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, communication_sub_type: string, value: string, last_updated: timestamp, activity_id: bigint, mobile: string]

In [26]:
contact_df_x1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_system_name', 'source_system_customer_id', 'communication_sub_type', 'value', 'last_updated', 'activity_id', 'mobile']

In [27]:
conditions = [lower(trim(contact_df.value))==lower(trim(agent_email.txt_email))]

contact_df_x2 = contact_df_x1.join(agent_email,conditions,how="left")
##contact_df_recheck_x1 = contact_df_recheck.filter(col('value').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
contact_df_x2.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_system_name', 'source_system_customer_id', 'communication_sub_type', 'value', 'last_updated', 'activity_id', 'mobile', 'txt_email']

In [29]:
contact_df_x2.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, communication_sub_type: string, value: string, last_updated: timestamp, activity_id: bigint, mobile: string, txt_email: string]

In [30]:
contact_df_x2=contact_df_x2.filter(col('value').isNotNull()).filter(col('source_system_customer_id').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
contact_df_x2_cnt = contact_df_x2.filter((contact_df_x2.value == contact_df_x2.mobile) | (contact_df_x2.value == contact_df_x2.txt_email)).filter(col('last_updated') <= progress_contact_mastercraft ).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select('source_system_customer_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
contact_df_x2_cnt.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [ ]:
contact_df_x2_cnt.count() ## 64772 ## 65111 (22 jan)

In [35]:
contact_df_x2_master = contact_df_x2.filter(col('last_updated') <= progress_contact_mastercraft ).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select('source_system_customer_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
contact_df_x2_master.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [37]:
contact_df_x2_cnt_new_rechk = contact_df_x2_master.exceptAll(contact_df_x2_cnt)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
contact_df_x2_cnt_new_rechk.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [ ]:
contact_df_x2_cnt_new_rechk.count()  ## 87122660 ## 87159770 (22 jan)

In [ ]:
contact_df_x2_cnt_new.distinct().count() ##87939785

In [ ]:
value_agent = contact_df_x2_cnt.agg({"source_system_customer_id": "count"}).collect()[0][0]

In [ ]:
value_agent ## 64772

In [ ]:
contact_cust_id_cnt = value_contactblity_master - value_agent 

In [ ]:
contact_cust_id_cnt ## 87122660  df_mastercraft_cnt ### 87127000

In [ ]:
contact_df_x2_cnt_rechk = contact_df_x2.filter((contact_df_x2.value != contact_df_x2.mobile) | (contact_df_x2.value != contact_df_x2.txt_email)).filter(col('last_updated') <= progress_contact_mastercraft ).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select('source_system_customer_id').distinct()

In [ ]:
contact_df_x2_cnt_rechk.count()

In [ ]:
from pyspark.sql.window import Window

contact_df = contact_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy("source_system_name","source_system_customer_id","activity_id")
        .orderBy(F.col("last_updated").desc())))

contact_df = contact_df.filter(F.col("rn") == 1).drop("rn")

In [ ]:
contact_df = contact_df.filter(~(col("value").isin(['1234567890','9876543210','9999988888','9090909090','9000000000','9900000000'])))
contact_df = contact_df.filter(~(lower(trim(col("value"))).isin(['abc@gmail.com','xyz@gmail.com','123@gmail.com','abc@d.com','abcd@gmail.com','null'])))

In [ ]:
contact_df.repartition('value')

In [ ]:
condition = [lower(trim(contact_df.value))==lower(trim(agent_no.mobile))]

contact_df = contact_df.join(agent_no,condition,how="left")
contact_df = contact_df.filter(col('mobile').isNull()).drop(contact_df.mobile)

condition = [lower(trim(contact_df.value))==lower(trim(agent_email.txt_email))]

contact_df = contact_df.join(agent_email,condition,how="left")
contact_df = contact_df.filter(col('txt_email').isNull()).drop(contact_df.txt_email)

In [ ]:
contact_df=contact_df.filter(col('value').isNotNull()).filter(col('source_system_customer_id').isNotNull())

In [ ]:
contact_df_cnt = contact_df.filter(col('last_updated') <= progress_contact_mastercraft).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select("source_system_customer_id").distinct()

In [ ]:
contact_df_cnt.count() ##87155715

In [ ]:
## mastercraft se count aaya = df_masterCraft_cnt.count() ## 87127698

### CUSTOMER_PROFILE

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.progress",
         "server.port":"1150-1170"}


progress_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.selectExpr('table_name','source','to_datetime','from_datetime')#\
#.filter(to_date(col('start_time'))==current_date())

In [ ]:
progress_df.persist()

In [ ]:
group_cols = ['table_name','source']

In [ ]:
progress_df1 = progress_df.groupby(group_cols).agg(max('to_datetime').alias('max_to_datetime')) 

In [ ]:
## for creating variable for max(to_datetime) of customer_profile table, to find out count of distinct GID in Masterfcraft table and customer_profile table by using as filter this datetime variable

progress_cust_prof = progress_df1.filter(col('table_name').isin('customer_profile')).collect()[0][1]

In [ ]:
progress_cust_prof

In [ ]:
col_str=['golden_id','last_updated']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_profile"}
profile_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
profile_df.persist()

In [ ]:
profile_df01 = profile_df.filter(col('last_updated') <= progress_cust_prof).select("golden_id","last_updated" ).distinct()

In [ ]:
profile_df01.count()

In [ ]:
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1160",
"dbschema": "public",
"dbtable": "mastercraft_bulk_detail_prod"}
master_bulk_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr('source_customer_id','dc_unified_id','dc_action_date').filter((col('dc_unified_id').isNotNull()))


In [ ]:
gscPythonOptions = {
    "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
    "user": "gpspark",
    "password": "spark@456",
    "dbschema": "public",
    "dbtable":"mastercraft_master_prod", 
    "partitionColumn":"row_num",
    "server.port":"1106",
    "partitions":6} 

master_prod_df=sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('dc_unified_id','dc_active_flag').filter((col('dc_unified_id').isNotNull()))\
.filter((col("dc_active_flag")==1))

In [ ]:
master= master_bulk_df.join(master_prod_df, 'dc_unified_id', "inner")

In [ ]:
# from pyspark.sql.window import Window

# master = master.withColumn("rn", F.row_number()
#         .over(Window.partitionBy(['dc_unified_id'])
#         .orderBy(F.col("dc_action_date").desc())))

# master = master.filter(F.col("rn") == 1).drop("rn")

In [ ]:
master.persist()

In [ ]:
master_df01 = master.filter(col('dc_action_date') <= progress_cust_prof).select("dc_unified_id").distinct()

In [ ]:
master_df01.count()

### CUSTOMER_DEMOGRAPHICS

In [ ]:
## for creating variable for max(to_datetime) of customer_profile table, to find out count of distinct GID in Masterfcraft table and customer_profile table

progress_cust_demog = progress_df1.filter(col('table_name').isin('customer_demographics')).collect()[0][1]

In [ ]:
progress_cust_demog

In [ ]:
col_str=['golden_id','last_updated']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_demographics"}
demog_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
demog_df1 = demog_df.filter(col('last_updated') <= progress_cust_demog).select("golden_id").distinct()

In [ ]:
demog_df1.count()

In [ ]:
## 1. cust_demo left join master_prod where active flag != 0
##check  distinct count gid 

## 2. master_bulk inner join master_prod where actice flag = 1
##check  distinct count  dc_unified_id


In [ ]:
demog_prod = demog_df.join(master_prod_df, master_prod_df.dc_unified_id == demog_df.golden_id ,'left')

In [ ]:
demog_prod = demog_prod.filter(col("dc_active_flag") == 1 )

In [ ]:
demog_prod_01 =  demog_prod.filter(col('last_updated') <= progress_cust_demog).select("golden_id").distinct()

In [ ]:
demog_prod_01.count()

In [ ]:
master_df_demo = master.filter(col('dc_action_date') <= progress_cust_demog).select("dc_unified_id").distinct()

In [ ]:
master_df_demo.count()

## ID

In [ ]:
## for creating variable for max(to_datetime) of customer_profile table, to find out count of distinct GID in Masterfcraft table and customer_profile table

progress_id = progress_df1.filter(col('table_name').isin('id')).collect()[0][1]

In [ ]:
progress_id

In [ ]:
col_str=['source_system_customer_id','last_updated_date']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "id"}
id_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
# id_df = id_df.withColumn('source_system_customer_id',trim(col('source_system_customer_id').cast('int'))).dropDuplicates()

In [ ]:
id_df1 = id_df.filter(col('last_updated_date') <= progress_id).select("source_system_customer_id").distinct()

In [ ]:
id_df1.count()

In [ ]:
id_df1.columns

In [ ]:
col_str=['source_id','source_customer_id','dc_unified_id','dc_action_date','pan','pan_is_verified',
        'cleansed_voterid','voter_id_is_verified', 'driving_license','driving_license_is_verified','passport','voter_id',
        'cleansed_passport','passport_is_verified']

master_bulk_df=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod","public")
master_bulk_df=master_bulk_df.filter((col('dc_unified_id').isNotNull()))\
                        .filter((col('source_customer_id').isNotNull())).filter((col('source_id').isNotNull()))

In [ ]:
# id= master_bulk_df.withColumn('source_customer_id',trim(col('source_customer_id').cast('int'))).dropDuplicates()

In [ ]:
id=master_bulk_df.filter((col('dc_unified_id').isNotNull()) & (col('source_id').isNotNull()))

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(col('pan')))\
.withColumn('dc_action_date', col('dc_action_date'))\
.withColumn('id_value', col('pan'))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id1=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('id_value'),col('dc_action_date'),\
                         col('primary_key'))
id1 = id1.withColumn('id_value',when(col('id_value').rlike("[A-Z]{5}[0-9]{4}[A-Z]{1}"),col('id_value'))
                                 .otherwise(lit(None)))

id1=id1.filter(col("id_value").isNotNull())

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(coalesce('cleansed_voterid','voter_id')))\
.withColumn('dc_action_date', col('dc_action_date'))\
.withColumn('id_value', coalesce('cleansed_voterid','voter_id'))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id2=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('id_value'),col('dc_action_date'),\
                         col('primary_key'))

id2 = id2.withColumn('id_value',when(col('id_value').rlike("^[A-Z]{3}\d{7}$"),col('id_value'))
                                 .otherwise(lit(None)))

id2=id2.filter(col("id_value").isNotNull())

In [ ]:
id_f1= id2.union(id1)

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(col('driving_license')))\
.withColumn('dc_action_date', col('dc_action_date'))\
.withColumn('id_value', col('driving_license'))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id3=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                        col('id_value'),col('dc_action_date'),\
                        col('primary_key'))

id3 = id3.withColumn('id_value',when(col('id_value').rlike("^(([A-Z]{2}[0-9]{2})( )|([A-Z]{2}-[0-9]{2}))((19|20)[0-9][0-9])[0-9]{7}$"),col('id_value'))
                                 .otherwise(lit(None)))

id3=id3.filter(col("id_value").isNotNull())

In [ ]:
id_f2= id_f1.union(id3)

In [ ]:
id=id.withColumn('source_system_name', col('source_id'))\
.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))\
.withColumn('activity_id', hash(coalesce('cleansed_passport','passport')))\
.withColumn('dc_action_date', col('dc_action_date'))\
.withColumn('id_value', coalesce('cleansed_passport','passport'))\
.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

In [ ]:
id4=id.select(col('source_system_name'),col('source_system_customer_id'),col('activity_id'),\
                         col('id_value'),col('dc_action_date'),\
                         col('primary_key'))

id4 = id4.withColumn('id_value',when(col('id_value').rlike("^[A-Za-z][1-9]\\d\\s?\\d{4}[1-9]$"),col('id_value'))
                                 .otherwise(lit(None)))

id4=id4.filter(col("id_value").isNotNull())

In [ ]:
id_f3= id_f2.union(id4)

In [ ]:
from pyspark.sql.window import Window

id_f3 = id_f3.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("dc_action_date").desc())))

id_f3 = id_f3.filter(F.col("rn") == 1).drop("rn")

In [ ]:
id_f3.columns

In [ ]:
id_f4 = id_f3.filter(col('dc_action_date') <= progress_id).select("source_system_customer_id").distinct()

In [ ]:
id_f4.count()

In [ ]:
id_f4.columns

In [ ]:
id_df1.exceptAll(id_f4).count()

## HEADER

In [ ]:
progress_header = progress_df1.filter(col('table_name').isin('header')).collect()[0][1]

In [ ]:
progress_header

In [ ]:
col_str=['golden_id','source_system_name','mapping_date']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "header"}
header_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
header_df1 = header_df.filter((col('mapping_date') <= progress_header) & (col('source_system_name') != 'NA')).select("golden_id").distinct()

In [ ]:
header_df1.count()

In [ ]:
col_str=['source_id','source_customer_id','dc_unified_id','dc_action_date']

master_bulk_df=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod","public")
master_bulk_df=master_bulk_df.filter((col('dc_unified_id').isNotNull()))\
                        .filter((col('source_customer_id').isNotNull())).filter((col('source_id').isNotNull()))

In [ ]:
gscPythonOptions = {
    "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
    "user": "gpspark",
    "password": "spark@456",
    "dbschema": "public",
    "dbtable":"mastercraft_master_prod", 
    "partitionColumn":"row_num",
    "server.port":"1106",
    "partitions":6} 

master_prod_df=sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('dc_unified_id','dc_active_flag').filter((col('dc_unified_id').isNotNull()))

In [ ]:
master_df= master_bulk_df.join(master_prod_df, master_bulk_df.dc_unified_id == master_prod_df.dc_unified_id, "inner")\
.drop(master_prod_df.dc_unified_id)

In [ ]:
master_df_2 = master_df.filter(col('dc_action_date') <= progress_header).select("dc_unified_id").distinct()

In [ ]:
master_df_2.count()

## ADDRESS

In [ ]:
col_str=['source_system_customer_id','last_updated']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "address"}
address_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
## for creating variable for max(to_datetime) of address table, to find out count of distinct GID in Masterfcraft table and customer_profile table by filtering with max_todatetime variable

progress_address = progress_df1.filter(col('table_name').isin('address')).collect()[0][1]

In [ ]:
progress_address

In [ ]:
address_df1 = address_df.filter(col('last_updated') <= progress_address).select("source_system_customer_id").distinct()

In [ ]:
address_df1.count()

In [ ]:
col_str=['source_id','source_customer_id','dc_unified_id',
         'coalesce(pstincomingcity,psnt_address_city) as psnt_address_city',
         'coalesce(pstincomingstate,psnt_address_state) as psnt_address_state',
         'coalesce(pstincomingpin,psnt_address_zip) as psnt_address_zip',
         'coalesce(cmnincomingcity,comm_address_city) as comm_address_city' ,
         'coalesce(cmnincomingstate,comm_address_state) as comm_address_state',
         'coalesce(cmnincomingpin,comm_address_zip) as comm_address_zip',
         'coalesce(pmtincomingcity,pmt_address_city) as pmt_address_city',
         'coalesce(pmtincomingstate,pmt_address_state) as pmt_address_state',
         'coalesce(pmnincomingpin,pmt_address_zip) as pmt_address_zip',
         'dc_action_date']

master_bulk_df=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod","public")
spark_df=master_bulk_df.filter((col('dc_unified_id').isNotNull()) &(col('source_id').isNotNull())) 

##& (col('source_customer_id').isNotNull())

In [ ]:
spark_df=spark_df.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))

In [ ]:
spark_df.columns

In [ ]:
spark_df = spark_df.withColumn('psnt_address_zip',when(col('psnt_address_zip').rlike("^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"),\
                                                       col('psnt_address_zip')).otherwise(lit(None)))

In [ ]:
spark_df = spark_df.withColumn('comm_address_zip',when(col('comm_address_zip').rlike("^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"),\
                                                       col('comm_address_zip')).otherwise(lit(None)))

In [ ]:
spark_df = spark_df.withColumn('pmt_address_zip',when(col('pmt_address_zip').rlike("^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"),\
                                                       col('pmt_address_zip')).otherwise(lit(None)))

In [ ]:
# spark_df2=spark_df1.filter((col('pmt_address_city').isNotNull()) | (col('pmt_address_state').isNotNull()) | (col('pmt_address_zip').isNotNull()))

In [ ]:
df_psnt = spark_df.filter((col('psnt_address_city').isNotNull()) | (col('psnt_address_state').isNotNull()) | (col('psnt_address_zip').isNotNull()))

In [ ]:
df_pmt = spark_df.filter((col('pmt_address_city').isNotNull()) | (col('pmt_address_state').isNotNull()) | (col('pmt_address_zip').isNotNull()))

In [ ]:
df_union = df_psnt.union(df_pmt)

In [ ]:
df_comm=spark_df.filter((col('comm_address_city').isNotNull()) | (col('comm_address_state').isNotNull()) | (col('comm_address_zip').isNotNull()))

In [ ]:
df_union_1 = df_union.union(df_comm)

In [ ]:
df_union_2 = df_union_1.filter(col('dc_action_date') <= progress_address).select("source_system_customer_id").distinct()

In [ ]:
df_union_2.count()

In [68]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…